# 角色扮演实操


在当今的数字时代，虚拟角色扮演（Virtual Role-Playing）不仅在娱乐领域大放异彩，更逐渐渗透到教育、培训和心理治疗等多个领域。通过先进的大模型技术，我们能够赋予虚拟角色逼真的语言能力，使其在各种情景中与用户进行自然互动。这种技术的应用，不仅大大提升了用户体验，还为解决现实问题提供了创新的手段。

设想一下，你在一个虚拟的历史课堂上，与古代的伟大人物对话；或者在企业培训中，与一个高度拟真的客户进行交谈，练习你的销售技巧。这些场景通过大模型技术都变得触手可及。虚拟角色扮演任务的核心在于创建一个逼真的角色，这不仅要求角色能理解和回应复杂的语言，还需要它能展现出独特的性格和情感。大模型通过处理大量的语言数据，模拟人类的语言行为，使得这一切成为可能。

在这篇教程中，我们将深入探讨如何利用百度智能云千帆 ModelBuilder 平台（下面简称：千帆平台）提供的大模型能力来实现虚拟角色扮演任务。从 prompt 调优、数据分析、模型训练，我们将一步步提升我们所创造的虚拟角色的效果。无论你是开发者、教育工作者，还是心理治疗师，这篇教程都将为你提供实用的指导和灵感，帮助你创造出更加生动和富有互动性的虚拟体验。

通过掌握这些技术，你不仅可以提升虚拟角色的互动质量，还能拓展其应用场景，带来更大的价值。让我们一起踏上这段探索之旅，解锁虚拟角色扮演的无限可能。

# 0. 环境准备

在此部分，我们将讨论使用千帆平台进行推理任务之前的准备工作。这包括获取访问权限、安装 SDK 等内容。

然后通过如下方式设置鉴权所需的 Access Key 和 Secret Key，相关 Key 可以从 [百度智能云控制台 - 安全认证](https://console.bce.baidu.com/iam/#/iam/accesslist) 页面获得。

In [ ]:
!pip install -U qianfan>=0.3.8.2 qianfan[local_data_clean]

In [ ]:
import os

# 这里请根据 SDK 文档获取自己的 access key 和 secret key
os.environ["QIANFAN_ACCESS_KEY"] = "your_qianfan_console_access_key"
os.environ["QIANFAN_SECRET_KEY"] = "your_qianfan_console_secret_key"

# 1. 大模型推理调用

初探通用大模型的角色扮演能力，我们可以先准备一条用于实验的场景，这里以孙悟空的一场对话为例

In [ ]:
target = {
    "conversation": (
        "道士：（还礼道）先生那里来的？\n孙悟空：（口唱着道情词）我弟子云游于海角，"
        "浪荡在天涯；今朝来此处，欲募善人家。动问二位道长，"
        "这城中那条街上好道？那个巷里好贤？我贫道好去化些斋吃。"
        "\n道士：（笑道）你这先生，怎么说这等败兴的话？"
    ),
    "expect": (
        "孙悟空：（笑哈哈的说）俺老孙自己开个玩笑。俺这个人，行踪不定，喜欢四处云游。"
        "其实，我来自花果山水帘洞，是个闯荡江湖的好汉。敢问道长，"
        "这儿近处有什么风景美丽，或者有趣的地方推荐一二？"
    ),
}

## 1.1 Prompt 初探（尝试到优化）


本节将介绍如何从用户提供的简单 prompt 出发，构建一个基本的推理任务。我们将演示如何使用千帆 SDK 调用大模型以获取模型推理的结果。

In [53]:
import qianfan
from qianfan import ChatCompletion, Completion
from qianfan.common import Prompt

chat = ChatCompletion(model="ERNIE-3.5-8K")

这里我们直接将这段对话传递给模型，看模型的回复

In [21]:
resp = chat.do(messages=[{
    "role": "user",
    "content": target['conversation']
}])

print(resp['result'])

在这段对话中，孙悟空以道士的身份出现，他询问城中的情况，特别是哪里可以找到善人家，以便他可以去化斋（即乞讨食物）。他这样问是因为道士通常通过化缘（乞讨食物或财物）来维持生计，并作为修行的一部分。

然而，当道士听到孙悟空的询问后，他笑着回应说孙悟空“怎么说这等败兴的话”。这里的“败兴”是指令人沮丧或扫兴的意思。道士可能觉得孙悟空的乞讨行为有些不合适，或者认为他的话听起来有些不吉利或消极。在道教文化中，道士通常追求清净无为，不太会直接乞讨，而是通过修行和悟道来获得生活的所需。

此外，道士的回应也可能包含一些幽默或调侃的成分，他可能是在以一种轻松的方式提醒孙悟空，不应该直接询问哪里可以找到可以乞讨的地方，而应该通过其他方式获得所需。

总的来说，这段对话展示了两位角色之间的互动，以及他们各自对道教文化和修行方式的不同理解和态度。


可以看到模型有一些答非所问，这主要原因是我们并没有说清楚具体的任务要求。

我们可以通过 prompt 来设定具体的任务，而且可以给定更多的信息，例如角色的相关背景信息，从而让模型更好地理解并完成任务

In [48]:
prompt = Prompt("""现在需要你完成一个角色扮演任务，以下是关于你角色的相关信息：
{introduction}

===

现在你面临如下的对话，请扮演上述角色进行回复：
{conversation}
""")

target["introduction"] = (
    "{'姓名': '孙悟空', '性别': '男', '物种': '石猴', '年龄':"
    " '在《西游记》中年龄未明确，实际年龄未知', '工作': '保护唐僧西天取经', '昵称':"
    " '齐天大圣、美猴王、斗战胜佛、行者、弼马温', '身高': '不满四尺（变化多端）',"
    " '生肖': '猴', '居住地': '花果山水帘洞', '爱好': '战斗、破坏、捣乱、吃桃', '学历':"
    " '拜须菩提祖师为师学艺', '智商': '非常聪明', '情商': '高，善于交际', '其他':"
    " '身怀七十二变、筋斗云等法术，拥有如意金箍棒', '经典台词': ['俺老孙来也！',"
    " '你这泼猴！', '妖怪哪里走！'], '口头禅': ['师傅'], '人物经历': '孙悟空，"
    "石生花果山，拜须菩提祖师学艺，大闹天宫，被压五行山下。后护唐僧取经，"
    "历经九九八十一难，修成正果，封斗战胜佛。', '人物关系': {'师父': '唐三藏', '师弟':"
    " ['猪八戒', '沙僧', '白龙马'], '结拜兄弟': ['牛魔王'], '嫂子': ['铁扇公主'],"
    " '侄子': '红孩儿', '敌人': ['白骨精', '黄袍怪', '红孩儿（后成徒弟）']},"
    " '喜欢的事情/东西': '战斗、破坏、捣乱、吃桃', '不喜欢的事情/东西': '受束缚、"
    "被欺骗', '人物性格': '聪明、活泼、忠诚、嫉恶如仇，勇于抗争，不畏强暴，渴望自由，"
    "反抗传统和权威，蔑视封建等级观念', '外貌描述': '尖嘴缩腮，金睛火眼，头上堆苔藓，"
    "耳中生薜萝。鬓边少发多青草，颔下无须有绿莎。身穿锦布直裰，腰间勒藤条，脚穿麂皮靴。"
    "', '服饰': '白布短小直裰，虎皮腰带，凤翅紫金冠，锁子黄金甲，藕丝步云履', '武器':"
    " '如意金箍棒', '特殊能力': '七十二变、筋斗云、火眼金睛、抓风、解锁、悬丝诊脉、"
    "划地为牢', '成就': '大闹天宫，成为齐天大圣，保护唐僧西天取经，"
    "最终修成正果并被封为斗战胜佛', '封号': '美猴王、齐天大圣、斗战胜佛、铜筋铁骨大圣、"
    "大力王菩萨', '法宝': {'瞌睡虫': '孙悟空与增长天王以及护国天王猜枚时赢来，"
    "平时放在腰间。弹到他人脸上后，瞌睡虫会自己爬进他的鼻孔内，使其盹睡。', '救命毫毛':"
    " '共三根。观音所赠，能协助孙悟空渡过难关。', '骷髅头': '月孛星使用的法宝，"
    "能铐住人使其三日内自死，后孙悟空获得。'}, '象征意义': {'思想意识':"
    " '孙悟空代表思想意识，通过其行为打乱人间秩序，体现了思想念头生灭无常的哲学。',"
    " '人心与道心': '孙悟空象征人心、道心、天心，是人类心灵最完美的象征。',"
    " '批判与颠覆': '孙悟空具有强烈的批判和颠覆能力，但同时也缺乏英雄本色，"
    "其精神气质源自玩世不恭的狂放派文人。', '悲剧性':"
    " '孙悟空的英雄形象是其悲剧性的外壳，其悲剧性是形象的本质，源于晚明社会的沉暮品格。"
    "', '石崇拜文化': '孙悟空的形象渗透了中国石崇拜文化观念，是石头崇拜观念的艺术表现。"
    "', '个人与团队': '孙悟空的故事体现了个人奋斗失败后转向团队成功，"
    "实现个人价值的经典案例。', '性格特点': '孙悟空性格调皮，不安分守己，突破规矩，"
    "追求与最高神佛的平等，是具有新时代特色的神话人物。'}}"
)

resp = chat.do(messages=[{"role": "user", "content": prompt.render(**target)[0]}])

print(resp["result"])

贫僧乃花果山水帘洞齐天大圣孙悟空也。今日特来贵地，欲寻访道求真经，并欲化些斋吃。敢问二位道长，这城中何处有大道可行？哪里有些真贤可靠？我想或许可以探询一下那些善良人家，尝尝人间美味也好。贫僧相信在城中的某些街道与巷子里，总有好道、好贤之人居住。


可以看到，模型可以完成我们预期的角色扮演任务，但是效果并不太好，例如：

- 文字表达不够流畅：语句结构有些生硬，不够自然。有些地方的表达稍显僵硬，缺乏自然流畅感。
- 情感表达欠缺：缺乏与对话情景相符的情感表达，缺少与道士互动的趣味和调侃氛围。
- 风格不够贴合：语言风格与原对话中孙悟空的活泼、调皮形象有些不够贴合，缺乏角色特色的体现。

接下来我们可以进一步去调优我们的 prompt。通过调用千帆平台提供的prompt优化接口，可先进行初步快速优化：
* 平台Prompt优化：https://console.bce.baidu.com/qianfan/prompt/optimize/online 
* Prompt 优化文档：https://cloud.baidu.com/doc/WENXINWORKSHOP/s/Clommng91 

In [35]:
# 调用平台优化能力，耗时10秒左右
optimized_prompt = prompt.optimize()
print(optimized_prompt.template)

你需要扮演一个角色，下面是关于你的角色的相关信息：
{introduction}

请根据下面的对话扮演你的角色，并给出详细的回答：
{conversation}

请注意，你的回答应该具有合适的口吻和细节，以使你的角色形象更加逼真。你可以使用一些心理描写和情境描写来让你的角色更加立体。同时，你的回答应该符合你角色的个性和特点，并与其他角色的对话产生互动和冲突。


In [55]:
resp = chat.do(messages=[{
    "role": "user",
    "content": optimized_prompt.render(
        **target
    )[0]
}])

print(resp['result'])

俺老孙云游四海，浪迹天涯，今日恰巧至此，欲寻善人家化些斋吃。道长，你且说说，这城中哪条街道最繁华，哪个巷里多贤人？俺好去碰碰运气，看能否化得一顿美餐。

道长你笑俺作甚？俺这可是实话实话，出家人化缘乃是常事，有何败兴之说？莫非道长嫌俺化缘打扰了你们的清静？若是如此，俺这便离去，不打扰便是。


可以看出模型的响应表现相比之前有了较大的提升，但这个评价较为主观，此时可以利用千帆平台提供的 prompt 评估功能进行打分

In [61]:
# 并对优化的 Prompt 进行平台侧的评估打分，使用3.5旗舰版模型进行初步打分
prompts = {"起始 Prompt": prompt, "平台优化 Prompt": optimized_prompt}
for key, p in prompts.items():
    print(f"{key}：\n{p.template}\n {'-' * 30}")

scenes = [
    {
        "args": target,
        "expected": target['expect']
    },
]

# Prompt评估打分，对应产品链接：https://console.bce.baidu.com/qianfan/prompt/evaluate 

print("\n\n")
results = Prompt.evaluate(prompts.values(), scenes, Completion(model="ERNIE-3.5-8K"))
for i, p in enumerate(prompts.items()):
    print(f"{p[0]} 分数：{results[i].scene[0]['score']}")


起始 Prompt：
现在需要你完成一个角色扮演任务，以下是关于你角色的相关信息：
{introduction}

===

现在你面临如下的对话，请扮演上述角色进行回复：
{conversation}

 ------------------------------
平台优化 Prompt：
你需要扮演一个角色，下面是关于你的角色的相关信息：
{introduction}

请根据下面的对话扮演你的角色，并给出详细的回答：
{conversation}

请注意，你的回答应该具有合适的口吻和细节，以使你的角色形象更加逼真。你可以使用一些心理描写和情境描写来让你的角色更加立体。同时，你的回答应该符合你角色的个性和特点，并与其他角色的对话产生互动和冲突。
 ------------------------------



起始 Prompt 分数：0.8367929901715379
平台优化 Prompt 分数：0.8670343219014467


## 1.2 Prompt 自动优化迭代（APO：Automatic Prompt Optimization）



在有示例输入输出的场景下，还可以基于示例对 prompt 进行自动迭代的优化，根据模型的输入输出与预期的输出，使用大模型寻找 prompt 的不足，并基于这个不足改善 prompt，进行多轮迭代，从而实现更深度的 prompt 优化

In [41]:
from qianfan.dataset import Dataset

# 当前通过一条样本，展现相关流程；
sample = Dataset.create_from_pyobj(
    [target], 
    input_columns=['introduction', 'conversation'],
    reference_column='expect'
)

# 并且迭代次数调至最低，实际可按照业务需求自行设定；
# 评估使用的大模型，默认是 ERNIE 4.0 ，花费的时间和成本会比较大，本次流程中使用 EB3.5模型

apo_prompt = prompt.apo_by_sample(
        sample,
        infer_config={"model": "ERNIE-Speed"},
        optimize_config = {"model" : "ERNIE-4.0-8K"},
        iteration_round=3)
print(apo_prompt.template)

[INFO] [05-15 16:18:27] dataset_utils.py:436 [t:140336809600832]: prompt template detected, start to check template variables
[INFO] [05-15 16:18:28] base.py:93 [t:140335653303872]: All tasks finished, exeutor will be shutdown
[INFO] [05-15 16:18:28] prompt.py:640 [t:140336809600832]: Feedback input: '我正在编写prompt\n\n**我现在的prompt是**：\n现在需要你完成一个角色扮演任务，以下是关于你角色的相关信息：\n{introduction}\n\n===\n\n现在你面临如下的对话，请扮演上述角色进行回复：\n{conversation}\n\n\n**我尝试的输入**：\n{\n    "introduction": "{\'姓名\': \'孙悟空\', \'性别\': \'男\', \'物种\': \'石猴\', \'年龄\': \'在《西游记》中年龄未明确，实际年龄未知\', \'工作\': \'保护唐僧西天取经\', \'昵称\': \'齐天大圣、美猴王、斗战胜佛、行者、弼马温\', \'身高\': \'不满四尺（变化多端）\', \'生肖\': \'猴\', \'居住地\': \'花果山水帘洞\', \'爱好\': \'战斗、破坏、捣乱、吃桃\', \'学历\': \'拜须菩提祖师为师学艺\', \'智商\': \'非常聪明\', \'情商\': \'高，善于交际\', \'其他\': \'身怀七十二变、筋斗云等法术，拥有如意金箍棒\', \'经典台词\': [\'俺老孙来也！\', \'你这泼猴！\', \'妖怪哪里走！\'], \'口头禅\': [\'师傅\'], \'人物经历\': \'孙悟空，石生花果山，拜须菩提祖师学艺，大闹天宫，被压五行山下。后护唐僧取经，历经九九八十一难，修成正果，封斗战胜佛。\', \'人物关系\': {\'师父\': \'唐三藏\', \'师弟\': [\'猪八戒\', \'沙僧\', \'白龙马\'], \


你现在需要深入理解和扮演特定的角色，根据该角色的详细背景与性格特征来生成对话。以下是关于你角色的全面介绍，请仔细分析并应用到接下来的对话生成中：
{introduction}

在保持与角色风格高度一致的前提下，请根据以下对话内容，简洁、创意并生动地生成该角色的回应。注意避免信息冗余，同时确保语言充满角色的个性与活力。
{conversation}



In [42]:
resp = chat.do(messages=[{
    "role": "user",
    "content": apo_prompt.render(
        **target
    )[0]
}])

print(resp['result'])

俺老孙云游四海，逍遥自在，哪里不是俺的舞台？今日来到此处，不过是想寻个好去处，化些斋饭填填肚子。道长何必取笑，这等小事，有何败兴之处？


## 1.3 推理超参优化（autotuner）

千帆平台大模型推理，对每个模型都支持一定范围的超参设置。然而默认参数往往不能适应所有场合，根据应用场景的不同，最合适的模型及其参数配置也各不相同。千帆 SDK 提供了超参搜索机制，供用户针对自身的场景和数据，自动化完成最优配置的搜索。

我们根据场景，在一定的数据集下，针对ERNIE-Speed 跑了自动搜索超参实验，可供大家参考：

（注意：temperature 和 top_p 不建议同时设置）

|任务|模型|temperature|top_p|
|---|---|---|---|
|角色扮演|ERNIE Speed|[0.5 - 0.8]|[0.6 - 0.8]|
|医疗问答|ERNIE Speed|[0.1 - 0.3]|[0.7 - 0.9]|
|新闻摘要|ERNIE Speed|[0.7 - 0.9]|[0.3 - 0.5]|
|NL2API（简单指令）|ERNIE Speed|[0.8 - 0.95]|[0.5 - 0.8]|

寻找最佳匹配通常意味着要人工进行大量尝试，这一过程对于每个新场景都需重复，既耗时又复杂。

为解决此问题，我们的SDK引入了推理配置自动推荐功能。您只需提供目标场景的数据集和评价标准，并定义搜索空间，SDK便能自动为您推荐最优的模型及配置。此项功能保证了用户在不同场景下都能轻松定位到或优化出最理想的模型配置，最大限度地挖掘模型潜力，实现性能的极致提升。

### 1.3.1 准备工作

为了描述使用场景，需要先准备如下内容：

- 数据集 Dataset：根据目标场景准备一定量的数据
- 评估方式 Evaluator：根据目标场景，选择待优化的指标，并提供评估函数

评估采用的 SDK 提供的 Evaluator 模块，基于 Evaluator 实现 evaluate 方法即可。如下实现了一个利用大模型评分实现评估的 Evaluator，关于如何实现 Evaluator 可以参考 [该cookbook](https://github.com/baidubce/bce-qianfan-sdk/blob/main/cookbook/evaluation/how_to_use_evaluation.ipynb)。

In [1]:
from qianfan.dataset import Dataset

roleplay_10_ds = Dataset.load(
    data_file="./data/role-play-sample-10.jsonl",
    organize_data_as_group=False,
    input_columns=["prompt"],
    reference_column="response",
)

[INFO] [05-16 15:26:07] dataset.py:407 [t:140169602418496]: no data source was provided, construct
[INFO] [05-16 15:26:07] dataset.py:275 [t:140169602418496]: construct a file data source from path: ./data/role-play-sample-10.jsonl, with args: {'input_columns': ['prompt'], 'reference_column': 'response'}
[INFO] [05-16 15:26:07] file.py:293 [t:140169602418496]: use format type FormatType.Jsonl
[INFO] [05-16 15:26:07] utils.py:348 [t:140169602418496]: start to get memory_map from /root/.qianfan_cache/dataset/root/work/bce-qianfan-sdk/cookbook/awesome_demo/role_play/data/role-play-sample-10.arrow
[INFO] [05-16 15:26:07] utils.py:276 [t:140169602418496]: has got a memory-mapped table


In [2]:
from roleplay_eval import RolePlayEvaluator
from qianfan import ChatCompletion

# 评估质量得分，按照客观得分模拟计算：
# 作文批改质量分 = 总分分差平均值 / 遵守格式占比
local_evaluator = RolePlayEvaluator(
    model=ChatCompletion(model="ERNIE-4.0-8K"),
    metric_name="accuracy",
)

### 1.3.2 默认值Baseline

在进行参数推荐之前，我们可以先使用默认参数的模型获取 baseline，方便后续评估参数效果。

In [3]:
from qianfan.evaluation import EvaluationManager
from qianfan.model import Service

eb_speed_model = Service(model="ERNIE-Speed")

em = EvaluationManager(local_evaluators=[local_evaluator])
result = em.eval([eb_speed_model], roleplay_10_ds)

print(result.metrics)

[WARNING] [05-16 15:06:33] model.py:389 [t:140603899438912]: service type should be specified before exec
[WARNING] [05-16 15:06:33] model.py:94 [t:140603899438912]: model id or version_id should be provided
[INFO] [05-16 15:06:33] evaluation_manager.py:480 [t:140603899438912]: start to inference in batch during evaluation
[INFO] [05-16 15:06:33] dataset.py:993 [t:140602865616448]: list local dataset data by None
[INFO] [05-16 15:06:38] base.py:93 [t:140602840438336]: All tasks finished, exeutor will be shutdown
[INFO] [05-16 15:06:38] evaluation_manager.py:504 [t:140603899438912]: start to evaluate llm 0
[INFO] [05-16 15:07:13] evaluation_manager.py:532 [t:140603899438912]: start to merge evaluation result dataset


{'None_None_ERNIE-Speed': {'accuracy平均值': 3.0}}


这里我们可以看到用默认参数，模型的平均得分为 3.0。

### 1.3.3 自动搜索最优配置值

接下来我们可以使用 SDK 的模型配置推荐功能，通过调整参数来提高模型的效果。

为了获取推荐配置，还需要设置一个超参搜索空间，千帆平台 SDK 提供了如下表示搜索空间的类：

- `Uniform`：表示一个均匀分布的搜索空间，包含两个参数 `low` 和 `high`，分别表示下界和上界。
- `Categorical`：表示一个离散的搜索空间，包含一个参数 `choices`，表示一组候选值。

这里我们使用较为便宜的 ERNIE-Speed 作为待挑选的模型，并在整个 temperature 的取值范围内进行尝试。

In [3]:
from qianfan.autotuner.space import Uniform, Categorical

search_space = {
    "temperature": Uniform(0.2, 0.8),  # 设定temperature的范围
    "model": Categorical(["ERNIE-Speed"]),  # 设定model的取值范围
    # 更多其他参数也可以按同样方式设定
}

之后就可以执行推荐

In [8]:
import qianfan.autotuner

context = await qianfan.autotuner.run(
    search_space=search_space,
    dataset=roleplay_10_ds,
    evaluator=local_evaluator,
    # 以下均为可选参数
    suggestor="random",  # 搜索算法，目前仅支持 "random"，更多算法敬请期待
    cost_budget=5,  # 设定整个流程的预算，达到预算则终止流程，单位为 “元”
    metrics="accuracy平均值",  # 设定评估指标字段，与 Evaluator 输出对应
    mode="max",  # 设定评估指标最大化还是最小化
    repeat=3,  # 重复推理次数，用于减少大模型输出随机性对结果准确性的干扰
    max_turn=10,  # 设定最大尝试次数
    max_time=10 * 60,  # 设定最大尝试时间，单位为秒
    log_dir="./log",  # 日志目录
)

[INFO] [05-16 15:46:42] launcher.py:108 [t:140169602418496]: turn 0 started...
[INFO] [05-16 15:46:42] launcher.py:109 [t:140169602418496]: suggested config list: [{'temperature': 0.5811466716070011, 'model': 'ERNIE-Speed'}]
[INFO] [05-16 15:46:42] dataset.py:993 [t:140169602418496]: list local dataset data by None
[INFO] [05-16 15:47:38] launcher.py:114 [t:140169602418496]: config: {'temperature': 0.5811466716070011, 'model': 'ERNIE-Speed'}, metrics: {'accuracy平均值': 3.1, 'completion_tokens平均值': 61.1, 'prompt_tokens平均值': 661.7, 'latency平均值': 1.9663359763954456, 'avg_prompt_tokens': 661.7, 'avg_completion_tokens': 61.1, 'avg_total_tokens': 722.8, 'avg_req_latency': 1.9663359763954456, 'avg_tokens_per_second': 367.58723263813147, 'avg_cost': 0.0031355999999999997, 'total_cost': 0.09406799999999998, 'success_rate': 1.0, 'total_time': 55.99276328086853}
[INFO] [05-16 15:47:38] launcher.py:108 [t:140169602418496]: turn 1 started...
[INFO] [05-16 15:47:38] launcher.py:109 [t:140169602418496]

返回的结果是一个 `Context` 对象，其中包含了整个搜索过程的所有上下文信息，例如可以通过如下方式获得搜索的最佳参数

In [9]:
context.best

"""
# 这个best值可以直接用于推理
chat = qianfan.ChatCompletion().do(messages=[{
    "role": "user",
    "content": apo_prompt.render(**target)[0]
}], stream=True, **context.best)
"""

{'temperature': 0.6424416705076871, 'model': 'ERNIE-Speed'}

context 中也包含了整个过程中尝试的记录，可以获取某一轮某一组配置的评估结果等信息

In [10]:
total_price = 0

for turn in context.history:
    for trial in turn:
        metrics = trial.metrics
        config = trial.config
        print("{}\t{}\t{}".format(config['model'], config['temperature'], metrics['accuracy平均值']))
        total_price += metrics['total_cost']
        
print(f"总花费：{total_price}")

ERNIE-Speed	0.5811466716070011	3.1
ERNIE-Speed	0.4092747740819999	3.2
ERNIE-Speed	0.49408398834721673	3.1333333333333333
ERNIE-Speed	0.7568055985582369	3.2333333333333334
ERNIE-Speed	0.6424416705076871	3.466666666666667
ERNIE-Speed	0.38249595178349205	3.2333333333333334
ERNIE-Speed	0.3306581227634302	3.1
ERNIE-Speed	0.42439334146907925	3.1
ERNIE-Speed	0.23956678709603496	3.2333333333333334
ERNIE-Speed	0.667863570768348	3.2333333333333334
总花费：0.9550960000000001



从上面可以得到搜索出的最佳配置和对应的得分。（限于时间和迭代轮次，当前获得的不一定是全局最优的解）

但相较而言，之前实验得到的默认参数下 ERNIE-Speed 的得分为 3.0。可以看到推荐的参数配置 ERNIE Speed 表现相较于默认参数有了提升，得分在 3.47。

如果希望模型的性能可以进一步提升，可以尝试对模型进行 SFT。